<a href="https://colab.research.google.com/github/SourabhMagadum/Hindi-Sentiment-Analysis/blob/master/Hindi_sentiment_analysis%20best.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
# from tensorflow.python.keras import Sequential
from keras.layers import  Dense, Embedding, LSTM, Bidirectional,GRU
# from tensorflow.python.keras.layers import Dense, Embedding, LSTM, GRU 
#
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import Sequential
from tensorflow.keras.activations import sigmoid
#
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from keras import optimizers
from keras import layers
# from tensorflow.python.keras import layers
import tensorflow as tf

In [3]:
import io
df = pd.read_csv('DATA.csv')


In [4]:
df.head()

,Sentence,Sentiment
0,"నాటింగ్‌హామ్‌ : దక్షిణాఫ్రికా, ఆస్ట్రేలియా వంట...",neutral
1,ఆసీస్‌ మ్యాచ్‌లో శతకంతో ఫామ్‌లోకి వచ్చిన శిఖర్...,neg
2,దీంతో జుట్టులో సమీ కరణాలు మారనున్నాయి.,neutral
3,నాలుగులో కుదు రుకున్న కేఎల్‌ రాహుల్‌ ఓపెనింగ్‌...,neutral
4,"రాహుల్‌, రోహిత్‌తో పాటుగా బరిలో దిగితే నాలుగో ...",neg


In [5]:
df.shape

(35142, 2)

In [6]:
y = df["Sentiment"]
X = df.drop(["Sentiment"], axis = 1)

In [7]:
# 70/15/15 train/test/val split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=1)

X_train.head()

,Sentence
910,"ఈ వన్డేలో స్మ_x005F_x007f_తి(90), మిథాలీ(63) క..."
129,ఈసారి సెర్బో బాస్నియన్‌ బాక్సర్‌ మిక్రో డ్రాజ్...
10392,ప్రపంచకప్‌లో భాగంగా ఈ నెల 30న ఇంగ్లండ్‌తో జరగన...
8515,శుక్రవారం మధ్యాహ్నం మూడు గంటలకు ముఖ్యమంత్రి నా...
10042,బర్మన్‌ రికార్డు


In [8]:
tk = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\]^_`{"}~\t\n')


tk.fit_on_texts(X['Sentence'])

# + 1 for unknown token
vocab_size = len(tk.word_index) +1

X_train_seq = tk.texts_to_sequences(X_train['Sentence'])
X_test_seq = tk.texts_to_sequences(X_test['Sentence'])
X_val_seq = tk.texts_to_sequences(X_val['Sentence'])
# Initializing max length of sentence to 20 words
max_length = 20

In [9]:
vocab_size

70969

In [10]:
tk.word_index

{'ఈ': 1,
 'కూడా': 2,
 'ఆ': 3,
 'నుంచి': 4,
 'చేశారు': 5,
 'సినిమా': 6,
 'అని': 7,
 'తన': 8,
 'అయితే': 9,
 'ఓ': 10,
 'ఆయన': 11,
 'చేసిన': 12,
 'ఉంది': 13,
 'ఉన్న': 14,
 'జగన్': 15,
 'రెండు': 16,
 'పార్టీ': 17,
 'కోసం': 18,
 'తొలి': 19,
 'పరుగులు': 20,
 'జట్టు': 21,
 'తెలిపారు': 22,
 'అన్నారు': 23,
 'కానీ': 24,
 'తర్వాత': 25,
 'పాటు': 26,
 'ఒక': 27,
 'మరో': 28,
 'చాలా': 29,
 'దీంతో': 30,
 'చేసింది': 31,
 'జరిగిన': 32,
 'తెలిసిందే': 33,
 'లో': 34,
 'చెందిన': 35,
 'చంద్రబాబు': 36,
 'భారత్\u200c': 37,
 'మంచి': 38,
 'మాట్లాడుతూ': 39,
 'భారీ': 40,
 'మంది': 41,
 'రూ': 42,
 'చెప్పారు': 43,
 'x005f': 44,
 'x007f': 45,
 'రెడ్డి': 46,
 'రెండో': 47,
 'మ్యాచ్\u200cలో': 48,
 '2': 49,
 'అనంతరం': 50,
 'విజయం': 51,
 'భారత': 52,
 'వచ్చిన': 53,
 'మూడు': 54,
 'అవకాశం': 55,
 'ఇక': 56,
 'కలిసి': 57,
 'చేసి': 58,
 'ఇప్పుడు': 59,
 'వరకు': 60,
 'ప్రభుత్వం': 61,
 'పరుగుల': 62,
 'జిల్లా': 63,
 'ముఖ్యమంత్రి': 64,
 'సీఎం': 65,
 'చేశాడు': 66,
 'అనే': 67,
 'మాత్రం': 68,
 'ఏపీ': 69,
 'మధ్య': 70,
 'ప్రస్తుతం': 71,
 'సం

In [11]:
print(vocab_size)


70969


In [12]:
X_train_seq

[[1,
  677,
  2585,
  44,
  45,
  2007,
  1391,
  1214,
  3773,
  57,
  2989,
  62,
  7109,
  1846,
  29389,
  326,
  712,
  811],
 [461, 28594, 28595, 4053, 28596, 28597, 2333, 8, 1087, 7946],
 [413, 164, 1, 262, 1193, 1923, 1522, 48, 37, 21, 82, 21130, 37574],
 [276, 667, 54, 340, 64, 1000, 36, 520, 20744],
 [5238, 377],
 [9710, 1283, 114, 1638, 2577, 2, 2166, 273, 1944, 712, 1499],
 [18263,
  11275,
  153,
  142,
  379,
  344,
  823,
  1106,
  189,
  66956,
  189,
  66957,
  2152,
  5294,
  66958,
  1175,
  11082,
  1253,
  66959,
  186,
  66960],
 [183, 1449, 753, 2992, 35, 36134, 36135, 103, 174, 103, 292, 120, 51, 500],
 [56, 27920, 424, 2808, 402, 16, 17728, 79, 341],
 [9, 517, 11, 4, 105, 464, 1488, 3, 713, 6, 664, 76],
 [331, 9073, 1, 385, 17811, 17075],
 [960, 25, 4143, 10442, 1443, 55905, 4269, 25177],
 [4862, 83, 412, 7212, 206, 495, 5148],
 [215, 29, 67290, 8, 18119, 552, 44, 45, 3795, 4049, 1, 3746, 2575, 1824],
 [135, 123, 3942, 1835, 427, 50, 757, 219, 794, 10263, 41463

In [13]:
#padding the sequences to make all the input sequences of the same length

X_train_seq_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_seq_pad = pad_sequences(X_test_seq, maxlen=max_length,padding='post')
X_val_seq_pad = pad_sequences(X_val_seq, maxlen=max_length,padding='post')


In [14]:
from tensorflow.keras.utils import to_categorical


In [15]:

le = LabelEncoder()
y_train_le = le.fit_transform(y_train)
y_test_le = le.transform(y_test)
y_val_le = le.transform(y_val)
y_train_oh = to_categorical(y_train_le)
y_test_oh = to_categorical(y_test_le)
y_val_oh = to_categorical(y_val_le)

In [16]:
y_train_le

array([2, 1, 1, ..., 1, 0, 0])

In [17]:
X_train_seq_pad

array([[    1,   677,  2585, ...,   811,     0,     0],
       [  461, 28594, 28595, ...,     0,     0,     0],
       [  413,   164,     1, ...,     0,     0,     0],
       ...,
       [   47,   156,    14, ...,     0,     0,     0],
       [  573,   193, 13848, ...,     0,     0,     0],
       [ 7762,  3528,   259, ...,     0,     0,     0]])

In [21]:
from tensorflow.keras.callbacks import EarlyStopping

In [22]:
emb_dims = 256
model = Sequential()
model.add(Embedding(vocab_size, emb_dims, input_length=max_length, embeddings_regularizer=tf.keras.regularizers.l2(0.0001)))
model.add(LSTM(units=16, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(3, activation='softmax'))
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
# Assuming X_train_seq_pad and y_train_oh are your training data (sequences and one-hot encoded labels)
history = model.fit(X_train_seq_pad, y_train_oh, epochs=32, batch_size=128,validation_data=(X_val_seq_pad, y_val_oh), shuffle=False)


Epoch 1/32
199/199 [==============================] - 101s 456ms/step - loss: 1.1230 - accuracy: 0.5429 - val_loss: 0.9362 - val_accuracy: 0.5896
Epoch 2/32
199/199 [==============================] - 90s 454ms/step - loss: 0.8653 - accuracy: 0.6592 - val_loss: 0.9849 - val_accuracy: 0.6333
Epoch 3/32
199/199 [==============================] - 90s 454ms/step - loss: 0.6851 - accuracy: 0.8004 - val_loss: 1.1588 - val_accuracy: 0.6282
Epoch 4/32
199/199 [==============================] - 91s 460ms/step - loss: 0.5712 - accuracy: 0.8694 - val_loss: 1.3035 - val_accuracy: 0.6220
Epoch 5/32
199/199 [==============================] - 80s 402ms/step - loss: 0.4866 - accuracy: 0.9074 - val_loss: 1.3807 - val_accuracy: 0.6233
Epoch 6/32
199/199 [==============================] - 88s 440ms/step - loss: 0.4269 - accuracy: 0.9289 - val_loss: 1.4941 - val_accuracy: 0.6077
Epoch 7/32
199/199 [==============================] - 86s 432ms/step - loss: 0.3919 - accuracy: 0.9416 - val_loss: 1.5097 - val_a

In [25]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 256)           18168064  
                                                                 
 lstm_1 (LSTM)               (None, 16)                17472     
                                                                 
 dense_1 (Dense)             (None, 3)                 51        
                                                                 
Total params: 18185587 (69.37 MB)
Trainable params: 18185587 (69.37 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
results = model.evaluate(X_test_seq_pad, y_test_oh)
print('/n')
print('Test accuracy of word embeddings model: {0:.2f}%'.format(results[1]*100))

165/165 [==============================] - 3s 20ms/step - loss: 2.2668 - accuracy: 0.6119
/n
Test accuracy of word embeddings model: 61.19%


In [27]:
results = model.evaluate(X_val_seq_pad, y_val_oh)
print('/n')
print('Validation accuracy of word embeddings model: {0:.2f}%'.format(results[1]*100))

141/141 [==============================] - 3s 20ms/step - loss: 2.2620 - accuracy: 0.6175
/n
Validation accuracy of word embeddings model: 61.75%
